In [2]:
from newsapi import NewsApiClient
import  requests
import urllib.request
import csv
import pandas as pd 
import numpy as np
import json

### News API (https://newsapi.org/pricing)
* Developers Free edition News API: 
    * 1,000 requests per day 
    * 250 requests per 6 hours
* A query can buffer a max of 10,000 results deep: a pageSize is maximum 100 and hence we could loop up to page=99
{"status":"error","code":"maximumResultsReached","message":"A query can buffer a max of 10,000 results deep. You have requested page 100 at 100 per page, which breaches the 10,000 limit."}

In [5]:
#Reading CSV file jobTitle.csv that contains all tech topics/skills
jobTitle_df = pd.read_csv("jobTitle.csv") #jobTitle_df is adataframe containing column with a list of tech topics/skills

In [4]:
news_list_nested=[]
totalResults=[]
jobTitle_column=pd.DataFrame()
page_size=100

In [ ]:
import time
ts = time.time()
print (ts)
import datetime
dt = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
print (dt) #output: 2018-08-09T14:02:01

### Fetching News articles that mention an IT skill from jobTitle_df. 
News articles needs to be published in the date range from 1st of Sep till value 'dt' (9th of August was last time we fetched News data and that was at the time current date and time). 
Retrieved news articles needs to be writen in English  (Note: some of the articles writen in other languages will be fetched too in case thay containe som English phrases and sentences, but that is an issue with News API)

In [9]:
for i in range(jobTitle_df.shape[0]):
    parametars={'apiKey':'7940eb8945c247448842bf08ff9926fe','language':'en','pageSize':page_size,'from':'2018-07-01T00:00:01','to':dt,'sort_by':'popularity','q':jobTitle_df['jobTitle'][i]}
    r = requests.get("https://newsapi.org/v2/everything", params=parametars)
    news_data = r.json()
    news_list_nested.append(pd.DataFrame(news_data)['articles'])
    totalResults.append(pd.DataFrame(news_data)['totalResults']) #total number of news articles for that topic/it skill
    jobTitle_column = jobTitle_column.append({'jobTitle': jobTitle_df['jobTitle'][i]}, ignore_index=True)
    news_data=[]
#http://newsapi.org/v2/everything?q=css&apiKey=7940eb8945c247448842bf08ff9926fe&pageSize=100&page=1&from=2018-08-09T14:02:01&to=2018-08-24 22:48:25&sort_by=publishedAt&language=en

#### Trending Tech Topics on News Portals: 
In order to determine what are the trending tech topics in the news we need to be able to fetch all possible news articles provided via News API. Page Size parameter is 100, while query can buffer a max of 10,000 results deep. Hence we could loop up to page=99. In order to fetch all the news articles available via News API we had to calculate number of pages required to extract all the news articles. We did that by using ceiling division operator and totalResults value fetched during first api request. In this way we were able to extract all possible news articles for a particular skill from multiple sources provided via News API (note we couldn't go over 10,000 articles because API limitation, hence the solution to do News API data refresh in smaller data frames).

In [ ]:
for i in range(jobTitle_df.shape[0]):
    number_of_pages=(totalResults[i][0]+100-1)//100  #ceiling division operator; pageSize=100 (pageSize=number of articles on page)
    print('topic number:',i+1, '    page=1,   number of pages:',number_of_pages)
    if number_of_pages>1:
        print('Number of pages:',number_of_pages,'>1')
        if number_of_pages>100:
            number_of_pages=100 #A query can buffer a max of 10,000 results deep: a pageSize is maximum 100 and hence we could loop up to page=100
        for j in range (1,number_of_pages):
            print('page=',j+1)
            if j==99:
                page_size=99 #A query can buffer a max of 10,000 results deep: a pageSize is maximum 100 and hence we could loop up to page=100 and pageSize needs to be changed to 99 so it does not reach 10000 articles limit
            parametars={'apiKey':'3a46087908b947ac9975975581ecf765','language':'en','pageSize':page_size,'page':'j+1','from':'2018-07-01T00:00:01','to':dt,'sort_by':'popularity','q':jobTitle_df['jobTitle'][i]} #had to regenerate api key for new calls https://newsapi.org/account
            r = requests.get("https://newsapi.org/v2/everything", params=parametars)
            news_data = r.json()
            news_list_nested.append(pd.DataFrame(news_data)['articles']) #loop
            totalResults.append(pd.DataFrame(news_data)['totalResults']) 
            jobTitle_column = jobTitle_column.append({'jobTitle': jobTitle_df['jobTitle'][i]}, ignore_index=True)
            news_data=[]
        page_size=100     

topic number: 1     page=1,   number of pages: 4
Number of pages: 4 >1
page= 2
page= 3
page= 4
topic number: 2     page=1,   number of pages: 10
Number of pages: 10 >1
page= 2
page= 3
page= 4
page= 5
page= 6
page= 7
page= 8
page= 9
page= 10
topic number: 3     page=1,   number of pages: 1
topic number: 4     page=1,   number of pages: 3
Number of pages: 3 >1
page= 2
page= 3
topic number: 5     page=1,   number of pages: 3
Number of pages: 3 >1
page= 2
page= 3
topic number: 6     page=1,   number of pages: 4
Number of pages: 4 >1
page= 2
page= 3
page= 4
topic number: 7     page=1,   number of pages: 10
Number of pages: 10 >1
page= 2
page= 3
page= 4
page= 5
page= 6
page= 7
page= 8
page= 9
page= 10
topic number: 8     page=1,   number of pages: 120
Number of pages: 120 >1
page= 2
page= 3
page= 4
page= 5
page= 6
page= 7
page= 8
page= 9
page= 10
page= 11
page= 12
page= 13
page= 14
page= 15
page= 16
page= 17
page= 18
page= 19
page= 20
page= 21
page= 22
page= 23
page= 24
page= 25
page= 26
pag

In [260]:
#Adding tach_topic and number_of_articles values to the news_list_nested list
for i in range(len(news_list_nested)):
    for j in range (len(news_list_nested[i])):
        news_list_nested[i][j].update({'tech_topic': jobTitle_column['jobTitle'][i], 'number_of_articles':totalResults[i][0]})

In [261]:
news_list_nested[4000][90]

{'author': 'InMoment',
 'description': 'SR. UI / UX DESIGNER\r\n&nbsp;\r\nWHAT WE ARE LOOKING FOR\r\nInMoment is searching for an experienced and talented UI/UX Designer. The designer will be responsible for evolving our platform by creating unique, user-centric layouts, prototypes, and interactions. The…',
 'number_of_articles': 675,
 'publishedAt': '2018-07-12T22:14:41Z',
 'source': {'id': None, 'name': 'Bamboohr.co.uk'},
 'tech_topic': 'Mobile User Experience',
 'title': 'InMoment is hiring a Senior UX / UI Designer in Mississauga',
 'url': 'https://inmoment.bamboohr.co.uk/jobs/view.php?id=150',
 'urlToImage': 'https://1fe31d82136e2caa7828-d44e7b307c294ec43d95ffbace54fbef.ssl.cf3.rackcdn.com/logos/cropped.jpg?v=27'}

In [262]:
#Flattening the news list
news_list_flatten = []
for i in range(len(news_list_nested)):
    for j in range (len(news_list_nested[i])):
        news_list_flatten.append(news_list_nested[i][j])
print(len(news_list_flatten))

2005141


In [263]:
#Creating news_df datagrame and populating it with data from news_list_flatten
news_df = pd.DataFrame()
news_df=news_df.append(news_list_flatten)
news_df.head()

,author,description,number_of_articles,publishedAt,source,tech_topic,title,url,urlToImage
0,Steave Sherman,Making of Out Of the Ordinary Short film. Out ...,111,2018-07-03T08:02:00Z,"{'id': None, 'name': 'Cgmeetup.net'}",3D Animation,Comment on Making of Out Of the Ordinary by St...,http://www.cgmeetup.net/home/making-ordinary/,http://www.cgmeetup.net/home/wp-content/upload...
1,None,This was a gift to my amazing 3D Animation & V...,111,2018-07-02T19:50:43Z,"{'id': None, 'name': 'Deviantart.com'}",3D Animation,A Gift,https://awkwardgerbil.deviantart.com/art/A-Gif...,None
2,Diptra,NewTek LightWave 3D 2018.0.5 Build 3068 Win x6...,111,2018-07-03T06:51:59Z,"{'id': None, 'name': 'Cgpersia.com'}",3D Animation,NewTek LightWave 3D 2018.0.5 Build 3068 Win x64,https://cgpersia.com/2018/07/newtek-lightwave-...,None
3,None,Minotorc Warrior Low Poly Character Modeling b...,111,2018-07-03T07:51:54Z,"{'id': None, 'name': 'Deviantart.com'}",3D Animation,Minotorc Warrior Low Poly Character Modeling,https://www.deviantart.com/gameyan/art/Minotor...,https://img00.deviantart.net/60cc/i/2018/184/a...
4,alvmlg,Unity is the ultimate game development platfor...,111,2018-07-03T17:39:12Z,"{'id': None, 'name': 'Unity.com'}",3D Animation,Get current animation from from animator,https://answers.unity.com/questions/1525058/ge...,https://unity3d.com/files/images/ogimg.jpg


In [264]:
#Converting strings to date time format
news_df['publishedAt'] = pd.to_datetime(news_df['publishedAt'])

In [ ]:
#Spliting nested column Source into source_id and source
#Provjeri da li dobro splita ili ponavlja rez kao i sa CEO

split_source = news_df['source'].apply(pd.Series) # split the source column from split_source
split_source = split_source.reset_index(drop=True) # reset index

news_df['source_id']=split_source['id']
news_df['source']=split_source['name']

news_df.head()

In [84]:
news_df=news_df.drop(['number_of_articles'], axis=1)

In [85]:
#Removing duplicate rows
news_df_no_duplicates = news_df.drop_duplicates(subset=None, keep='first', inplace=False)
news_df_no_duplicates 

,author,description,publishedAt,source,tech_topic,title,url,urlToImage,source_id
0,Steave Sherman,Making of Out Of the Ordinary Short film. Out ...,2018-07-03 08:02:00,Cgmeetup.net,3D Animation,Comment on Making of Out Of the Ordinary by St...,http://www.cgmeetup.net/home/making-ordinary/,http://www.cgmeetup.net/home/wp-content/upload...,None
1,None,This was a gift to my amazing 3D Animation & V...,2018-07-02 19:50:43,Deviantart.com,3D Animation,A Gift,https://awkwardgerbil.deviantart.com/art/A-Gif...,None,None
2,Diptra,NewTek LightWave 3D 2018.0.5 Build 3068 Win x6...,2018-07-03 06:51:59,Cgpersia.com,3D Animation,NewTek LightWave 3D 2018.0.5 Build 3068 Win x64,https://cgpersia.com/2018/07/newtek-lightwave-...,None,None
3,None,Minotorc Warrior Low Poly Character Modeling b...,2018-07-03 07:51:54,Deviantart.com,3D Animation,Minotorc Warrior Low Poly Character Modeling,https://www.deviantart.com/gameyan/art/Minotor...,https://img00.deviantart.net/60cc/i/2018/184/a...,None
4,alvmlg,Unity is the ultimate game development platfor...,2018-07-03 17:39:12,Unity.com,3D Animation,Get current animation from from animator,https://answers.unity.com/questions/1525058/ge...,https://unity3d.com/files/images/ogimg.jpg,None
5,PersianKiller,Unity is the ultimate game development platfor...,2018-07-01 06:45:26,Unity.com,3D Animation,set position inside animation window via scrip...,https://answers.unity.com/questions/1524115/se...,https://unity3d.com/files/images/ogimg.jpg,None
6,None,£6 - Amazon Amazon have dropped the price of m...,2018-07-04 15:04:57,Hotukdeals.com,3D Animation,104° - How To Train Your Dragon 3D + more £6 (...,https://www.hotukdeals.com/deals/how-to-train-...,https://images.hotukdeals.com/threads/thread_f...,None
7,Apple,Simlab Composer 9.0.2 Multilingual Windows x64...,2018-07-02 16:32:54,Gfxbing.com,3D Animation,Simlab Composer 9.0.2 Multilingual,http://gfxbing.com/pc-software/866953-simlab-c...,http://www.hostpic.org/images/1807022157260101...,None
8,Apple,Simlab Composer 9.0.3 Multilingual Windows x64...,2018-07-03 12:51:46,Gfxbing.com,3D Animation,Simlab Composer 9.0.3 Multilingual,http://gfxbing.com/pc-software/867139-simlab-c...,http://www.hostpic.org/images/1807031819220097...,None
9,None,First WIP.. Final screenshot of our project 3D...,2018-07-02 18:50:24,Deviantart.com,3D Animation,Screenshot,https://www.deviantart.com/pitanime80/art/Scre...,https://img00.deviantart.net/c28f/i/2018/183/d...,None


In [ ]:
#Replacing all the values 'Data Scientist' from the Column tech_topic with value 'Data Science' ,because
#Meetup tech topic only have value 'data science'. We wanted to retrieve articles containing 'data scientist', 
#hence we retrieved them and now replacing with value 'data science'
news_df_no_duplicates.tech_topic = news_df_no_duplicates.tech_topic.replace({"Data Scientist": "Data Science"})

In [89]:
#wrtiting it to csv
news_df_no_duplicates.to_csv('NewsData.csv', sep = ',', index = False) # create csv for interview_difficulty

## Keeping only exact match
### News API didn't retrieve exact match (eg. for .Net topic we retrieved articles about .Net but also about Net Wroth that we don't want)

In [101]:
len(news_df_no_duplicates)

117761

In [240]:
news_df_exact_match= news_df_no_duplicates

In [241]:
len(news_df_exact_match)

117761

In [242]:
#Removing all the news (rows) that have none value in the column 'description' and / or 'title
news_df_exact_match = news_df_exact_match.dropna(axis=0, subset=['description', 'title'])

In [243]:
len(news_df_exact_match)

116545

In [244]:
#list of row indexes that contain tech topic in description or title columns (exact match rows/news articles)
index = [] 

In [245]:
#Populating list index only with indexes of rows that contain value of thech topic in column 'description' or column 'titile'
for i in range(len(news_df_exact_match)):
    if (news_df_exact_match.iloc[i]['tech_topic'] in news_df_exact_match.iloc[i]['description'] or 
        news_df_exact_match.iloc[i]['tech_topic'] in news_df_exact_match.iloc[i]['title']) ==True:
        index.append(i)

In [246]:
len(index)

27098

In [251]:
#Keeping only rows that represent articles that contin tech topic either in description or title
news_df_exact_match=news_df_exact_match.iloc[index]

In [252]:
len(news_df_exact_match)

27098

In [253]:
news_df_no_duplicates.shape 

(117761, 9)

In [254]:
news_df.shape

(1855848, 9)

In [255]:
news_df_exact_match.shape

(27098, 9)

In [256]:
#wrtiting it to csv
news_df_exact_match.to_csv('newsData_Exact_match.csv', sep = ',', index = False) # create csv for interview_difficulty

Future scope: 
* write a code for api generation to avoid repated code